In [1]:
import numpy as np
import pandas as pd
import math
import netCDF4 as nc
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import datetime

import pyart

import boto3
import botocore
from botocore.client import Config

from metpy.io import Level2File
from metpy.plots import add_timestamp, ctables

import cartopy.crs as ccrs
import netCDF4

import time
import os

#  32.5,  33.5
# -97.5, -96.5


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [66]:
folder = "spatial"
node_id = "10004098"
dir_out = "../data/"
dir_data = "../data/"
fn_data = dir_data + "driving_" + node_id + "_landsat.csv"

dir_nexrad = '/Volumes/Backup Plus/NEXRAD/data/'
vars = ["reflectivity", #  Reflectivity (dBZ)
        "spectrum_width", #  Spectrum Width (meters_per_second)
        "velocity", #  Mean doppler Velocity (meters_per_second)
        "differential_reflectivity", #  log_differential_reflectivity_hv (dB)
        "differential_phase", #  differential_phase_hv (degrees)
        "cross_correlation_ratio", #  Cross correlation_ratio (RHOHV) (ratio)
        "ROI", #  Radius of influence for mapping (m)
       ]

df = pd.read_csv(fn_data)
df['UTC'] = pd.to_datetime(df['UTC'])
df.head()

/Users/Strawberry/opt/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,UTC,Illuminance,360nm,361nm,362nm,363nm,364nm,365nm,366nm,367nm,...,Zenith,latitude,longitude,altitude,LocalTime,date,surface reflectance band 1,surface reflectance band 2,surface reflectance band 3,surface reflectance band 4
0,2020-01-08 16:43:40,3849.500854,0.042302,0.042491,0.043221,0.044318,0.045591,0.046490,0.047298,0.047995,...,61.113035,32.992275,-96.757422,226.80,2020-01-08 10:43:40,2020/01/08,882,995,1352,1564
1,2020-01-08 16:43:50,2345.220551,0.032529,0.032651,0.033183,0.033997,0.034946,0.035627,0.036239,0.036761,...,61.096021,32.992279,-96.756782,226.74,2020-01-08 10:43:50,2020/01/08,582,656,925,1029
2,2020-01-08 16:44:00,2385.540894,0.033112,0.033226,0.033758,0.034580,0.035541,0.036231,0.036851,0.037380,...,61.079027,32.991883,-96.756719,232.32,2020-01-08 10:44:00,2020/01/08,974,1096,1371,1635
3,2020-01-08 16:44:10,2566.076355,0.033985,0.034098,0.034641,0.035480,0.036464,0.037165,0.037798,0.038343,...,61.062055,32.991692,-96.757119,226.90,2020-01-08 10:44:10,2020/01/08,740,886,1265,1484
4,2020-01-08 16:44:20,2944.573364,0.040020,0.040161,0.040814,0.041823,0.043005,0.043871,0.044654,0.045327,...,61.045103,32.991671,-96.757908,231.32,2020-01-08 10:44:20,2020/01/08,1109,1267,1699,2060


In [67]:
features = ['Zenith', 'surface reflectance band 1', 'surface reflectance band 2', 'surface reflectance band 3', 'surface reflectance band 4']
for var in vars:
    for i in range(10+1):
        var_i = var + ' ' + str(i) + 'km';
        df[var_i] = None
        features.append(var_i)

/Users/Strawberry/opt/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# filename = dir_nexrad + '2020-01-08/2020_01_08_KFWS_KFWS20200108_191600_V06'
# radar = pyart.io.read(filename)


In [240]:
date_last = '00000000'
i_last = -1

i = 0
while i < len(df):
    row = df.loc[i]
    UTC, long, lat = row[['UTC','longitude','latitude']]
    # get the date of spatial data
    year  = '%02d' % UTC.year
    month = '%02d' % UTC.month
    day = '%02d' % UTC.day
    date_str = year+month+day
    date = datetime.datetime.strptime(date_str,'%Y%m%d').date()
    
    # on this date, filter spatial data
    df_day = df[df['UTC'].apply(lambda x:x.date()) == date]
    
    # on this date, find the dir of NEXRAD 
    dir_day = dir_nexrad + year+'-'+month+'-'+day+'/'
    
#     filename_last = '0000_00_00_KFWS_KFWS00000000_000000_V06'
    filenames = []
    times = np.array([])
    for filename in os.listdir(dir_day):
        if (filename[0] == '.') | (filename[-1] == 'M'):
            continue
        filenames.append(filename)
        times = np.append(times, pd.to_datetime(filename[-19:-4],format = '%Y%m%d_%H%M%S'))
    times = np.append(times, pd.to_datetime('20990101_010101',format = '%Y%m%d_%H%M%S'))
    
    times_threshold = times[:-1]+(times[1:]-times[:-1])/2
    
    # for each file in NEXRAD, fill into df
    for j in range(len(filenames)):
        t_threshold = times_threshold[j]
        filename = dir_day + filenames[j]
        
        df_iwant = df_day[df_day['UTC']<=t_threshold] # time filter
        if len(df_iwant) == 0:
            continue
        
        # read NEXRAD data
        radar = pyart.io.read(filename)
        # find NEXRAD data from longitude, latitude
        grid = pyart.map.grid_from_radars(radar, grid_shape=(11, 201, 201),\
                              grid_limits=((0.,10000,), (-100000., 100000.), (-100000, 100000.)))

            
            
        break
        df_day = df_day[df_day['UTC']>t_threshold]
    break
    
    # get the time
    hour  = t[:2]
    minute = t[2:-2]
    second = t[-2:]
    
    t = hour+minute+second
    # find the filename most close to this time
    
    
    
    
    
    i += 1
    

/Users/Strawberry/opt/miniconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:


filename = ['/Volumes/Backup Plus/NEXRAD/halfyear/2020-01/2020-01-01/000000/2020-01-01_000000.nc'];
origin_latitude = ncread(filename, 'origin_latitude');
origin_longitude = ncread(filename, 'origin_longitude');
xs0 = ncread(filename, 'x');
ys0 = ncread(filename, 'y');
dx = xs0(2) - xs0(1);
dy = ys0(2) - ys0(1);
R = 6371*1000;


year = '2020';
for i_month = [1:3]
    month = num2str(i_month,'%02d');
    year_month = [year, '-', month];
    
    iwant_month = (df.UTC.Month == i_month);
    if sum(iwant_month) == 0
        continue;
    end
    
    for i_day = [1:31]
        day = num2str(i_day,'%02d');
        year_month_day = [year_month, '-', day];
        
        iwant_day = (iwant_month & (df.UTC.Day == i_day));
        if sum(iwant_day) == 0
            continue;
        end
        
        year_month_day

        for i_hour = [0:23]
            hour = [num2str(i_hour,'%02d'),'0000'];
            
            iwant_hour = (iwant_day & (df.UTC.Hour == i_hour));
            if sum(iwant_hour) == 0
                continue;
            end
            
            filename = ['/Volumes/Backup Plus/NEXRAD/halfyear/', year_month, '/', year_month_day, '/', hour, '/', year_month_day, '_', hour, '.nc'];
            
            lats = df.latitude(iwant_hour);
            lons = df.longitude(iwant_hour);
            
            xs = R*deg2rad(lons - origin_longitude) * cos(deg2rad(origin_latitude));
            ys = R*deg2rad(lats - origin_latitude);
            rows = (ys - ys0(1)) / dy + 1;
            cols = (xs - xs0(1)) / dx + 1;
            
            rows_floor = floor((ys - ys0(1)) / dy) + 1;
            cols_floor = floor((xs - xs0(1)) / dx) + 1;
            
            rows_ceil = ceil((ys - ys0(1)) / dy) + 1;
            cols_ceil = ceil((xs - xs0(1)) / dx) + 1;
            
            for i_var = 1:length(vars)
                var = vars(i_var);
                values = ncread(filename, var);
                if var ~= "ROI"
                    values = fillmissing(values,'constant',-9999.0);
                else
                    values = fillmissing(values,'constant',1e+20);
                end
                
                for i = 1:11
                    var_i = var + ' ' + int2str(i-1) + 'km';
                    
                    % Linear nterpolation
                    values_ff = values(sub2ind(size(values), rows_floor, cols_floor, ones(size(rows))*i));
                    values_fc = values(sub2ind(size(values), rows_floor, cols_ceil,  ones(size(rows))*i));
                    values_cf = values(sub2ind(size(values), rows_ceil,  cols_floor, ones(size(rows))*i));
                    values_cc = values(sub2ind(size(values), rows_ceil,  cols_ceil,  ones(size(rows))*i));
                    
                    values_f = (values_ff.*(cols-cols_ceil) + values_fc.*(cols-cols_floor))./(cols_ceil - cols_floor);
                    values_c = (values_cf.*(cols-cols_ceil) + values_cc.*(cols-cols_floor))./(cols_ceil - cols_floor);
                    
                    df.(var_i)(iwant_hour) = (values_f.*(rows-rows_ceil) + values_c.*(rows-rows_floor))./(rows_ceil - rows_floor);
                    
                end
            end
        end
    end
end

fn_out = dir_data + "driving_" + node_id + "_landsat_NEXRAD.csv";
writetable(df,fn_out)